# 00_core_validations
=========================

**Objetivo:** Validar cada etapa del pipeline de transformación

1. RAW → Carga inicial
2. Deduplicación de snapshots
3. Filtro temporal Q1 2025
4. JOIN loans ↔ customers (risk segments)
5. JOIN loans ↔ repayments (revenue)
6. Cálculo de revenue
7. Agregación por cohorts
8. Validación final

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Setup paths
base_path = Path.cwd()
while 'analisis_adhoc' in str(base_path):
    base_path = base_path.parent

raw_path = base_path / 'data' / 'raw'
exports_path = base_path / 'data' / 'exports'

print(f"📁 Working dir: {Path.cwd()}")
print(f"📂 Base path: {base_path}")
print(f"📍 Raw path: {raw_path} (existe: {raw_path.exists()})")
print(f"📍 Exports path: {exports_path} (existe: {exports_path.exists()})")

# Definir Q1 2025
Q1_PERIODS = [pd.Period('2025-01'), pd.Period('2025-02'), pd.Period('2025-03')]
print(f"\n🎯 Target: Q1 2025 = {[str(p) for p in Q1_PERIODS]}")

📁 Working dir: c:\Users\mijai\kueski-analytics-engineer-mijail\analisis_adhoc\data_validation
📂 Base path: c:\Users\mijai\kueski-analytics-engineer-mijail
📍 Raw path: c:\Users\mijai\kueski-analytics-engineer-mijail\data\raw (existe: True)
📍 Exports path: c:\Users\mijai\kueski-analytics-engineer-mijail\data\exports (existe: True)

🎯 Target: Q1 2025 = ['2025-01', '2025-02', '2025-03']


---
## 1️⃣ VALIDACIÓN: CARGA RAW
¿Se cargaron todos los archivos correctamente?

In [3]:
print("="*80)
print("1️⃣ VALIDACIÓN: CARGA RAW")
print("="*80)

# Cargar archivos raw
loans_raw = pd.read_csv(raw_path / 'AE_challenge_loans.csv')
repayments_raw = pd.read_csv(raw_path / 'AE_challenge_repayments.csv')
customers_raw = pd.read_csv(raw_path / 'AE_challenge_customer.csv')

print(f"\n📥 ARCHIVOS RAW CARGADOS:")
print(f"   loans.csv:      {len(loans_raw):>10,} filas | {loans_raw['loan_id'].nunique():,} loan_ids únicos")
print(f"   repayments.csv: {len(repayments_raw):>10,} filas | {repayments_raw['loan_id'].nunique():,} loan_ids únicos")
print(f"   customers.csv:  {len(customers_raw):>10,} filas | {customers_raw['user_id'].nunique():,} user_ids únicos")

print(f"\n📋 COLUMNAS:")
print(f"   loans:      {loans_raw.columns.tolist()}")
print(f"   repayments: {repayments_raw.columns.tolist()}")
print(f"   customers:  {customers_raw.columns.tolist()}")

print(f"\n✅ RAW cargado correctamente")

1️⃣ VALIDACIÓN: CARGA RAW

📥 ARCHIVOS RAW CARGADOS:
   loans.csv:         202,294 filas | 29,222 loan_ids únicos
   repayments.csv:     91,296 filas | 26,497 loan_ids únicos
   customers.csv:       4,500 filas | 4,500 user_ids únicos

📋 COLUMNAS:
   loans:      ['loan_id', 'user_id', 'disbursed_date', 'limit_month', 'term', 'interestrate', 'original_schedule', 'requested_amount', 'founded_amount', 'cogs_total_cost', 'charge_off', 'capital_balance', 'delinquency_status']
   repayments: ['user_id', 'loan_id', 'event_date', 'amount_trans', 'principalamount_trans', 'interestamount_trans', 'feesamount_trans', 'penaltyamount_trans', 'taxoninterestamount_trans', 'taxonfeesamount_trans', 'taxonpenaltyamount_trans', 'repayment_transaction_id']
   customers:  ['user_id', 'acquisition_date', 'acquisition_cost', 'channel', 'risk_band_production', 'city', 'state']

✅ RAW cargado correctamente


---
## 2️⃣ VALIDACIÓN: DEDUPLICACIÓN DE SNAPSHOTS
El archivo loans tiene múltiples snapshots por préstamo. ¿Cuántos duplicados hay?

In [4]:
print("="*80)
print("2️⃣ VALIDACIÓN: DEDUPLICACIÓN DE SNAPSHOTS")
print("="*80)

filas_totales = len(loans_raw)
loans_unicos = loans_raw['loan_id'].nunique()
snapshots_duplicados = filas_totales - loans_unicos
ratio_duplicacion = filas_totales / loans_unicos

print(f"\n📊 ANÁLISIS DE DUPLICADOS:")
print(f"   Filas totales:           {filas_totales:,}")
print(f"   Loan IDs únicos:         {loans_unicos:,}")
print(f"   Snapshots duplicados:    {snapshots_duplicados:,}")
print(f"   Ratio de duplicación:    {ratio_duplicacion:.1f}x")

# Ver distribución de snapshots por loan
snapshots_por_loan = loans_raw.groupby('loan_id').size()
print(f"\n📈 DISTRIBUCIÓN DE SNAPSHOTS POR LOAN:")
print(f"   Mínimo:   {snapshots_por_loan.min()}")
print(f"   Promedio: {snapshots_por_loan.mean():.1f}")
print(f"   Máximo:   {snapshots_por_loan.max()}")

print(f"\n   Top 5 loans con más snapshots:")
print(snapshots_por_loan.nlargest(5))

print(f"\n✅ Se deben eliminar {snapshots_duplicados:,} filas duplicadas")
print(f"   Resultado esperado: {loans_unicos:,} filas (1 por loan)")

2️⃣ VALIDACIÓN: DEDUPLICACIÓN DE SNAPSHOTS

📊 ANÁLISIS DE DUPLICADOS:
   Filas totales:           202,294
   Loan IDs únicos:         29,222
   Snapshots duplicados:    173,072
   Ratio de duplicación:    6.9x

📈 DISTRIBUCIÓN DE SNAPSHOTS POR LOAN:
   Mínimo:   1
   Promedio: 6.9
   Máximo:   12

   Top 5 loans con más snapshots:
loan_id
1576357023369715555    12
1576359652044889991    12
1576364328080645905    12
1576364932782677983    12
1576366881522536687    12
dtype: int64

✅ Se deben eliminar 173,072 filas duplicadas
   Resultado esperado: 29,222 filas (1 por loan)


---
## 3️⃣ VALIDACIÓN: FILTRO TEMPORAL Q1 2025
¿Cuántos préstamos hay por mes? ¿Cuántos quedan después de filtrar a Q1?

In [5]:
print("="*80)
print("3️⃣ VALIDACIÓN: FILTRO TEMPORAL Q1 2025")
print("="*80)

# Preparar fechas
loans_raw['disbursed_date'] = pd.to_datetime(loans_raw['disbursed_date'])
loans_raw['vintage_month'] = loans_raw['disbursed_date'].dt.to_period('M')

# Distribución completa por mes (usando loan_ids únicos)
distribucion_mensual = loans_raw.groupby('vintage_month')['loan_id'].nunique().sort_index()

print(f"\n📅 DISTRIBUCIÓN COMPLETA POR MES (préstamos únicos):")
for mes, count in distribucion_mensual.items():
    marca = "← Q1" if mes in Q1_PERIODS else ""
    print(f"   {mes}: {count:>6,} {marca}")

# Calcular totales
loans_q1 = loans_raw[loans_raw['vintage_month'].isin(Q1_PERIODS)]['loan_id'].nunique()
loans_otros = loans_raw[~loans_raw['vintage_month'].isin(Q1_PERIODS)]['loan_id'].nunique()

print(f"\n📊 RESUMEN:")
print(f"   Q1 2025 (Ene-Mar):  {loans_q1:,} préstamos ({loans_q1/loans_unicos*100:.1f}%)")
print(f"   Otros meses:        {loans_otros:,} préstamos ({loans_otros/loans_unicos*100:.1f}%)")

print(f"\n✅ Después del filtro Q1 2025 deben quedar: {loans_q1:,} préstamos")

3️⃣ VALIDACIÓN: FILTRO TEMPORAL Q1 2025

📅 DISTRIBUCIÓN COMPLETA POR MES (préstamos únicos):
   2025-01:  2,140 ← Q1
   2025-02:  3,694 ← Q1
   2025-03:  3,562 ← Q1
   2025-04:  2,699 
   2025-05:  2,786 
   2025-06:  2,277 
   2025-07:  2,294 
   2025-08:  2,087 
   2025-09:  1,800 
   2025-10:  1,978 
   2025-11:  2,155 
   2025-12:  1,750 

📊 RESUMEN:
   Q1 2025 (Ene-Mar):  9,396 préstamos (32.2%)
   Otros meses:        19,826 préstamos (67.8%)

✅ Después del filtro Q1 2025 deben quedar: 9,396 préstamos


---
## 4️⃣ VALIDACIÓN: JOIN LOANS ↔ CUSTOMERS
¿Todos los loans tienen un customer asociado? ¿Se preservan los risk segments?

In [8]:
print("="*80)
print("4️⃣ VALIDACIÓN: JOIN LOANS ↔ CUSTOMERS")
print("="*80)

# Obtener user_ids de loans Q1
loans_q1_df = loans_raw[loans_raw['vintage_month'].isin(Q1_PERIODS)].drop_duplicates('loan_id')
user_ids_loans = set(loans_q1_df['user_id'].unique())
user_ids_customers = set(customers_raw['user_id'].unique())

# Análisis de match
users_con_match = user_ids_loans & user_ids_customers
users_sin_match = user_ids_loans - user_ids_customers

print(f"\n📊 ANÁLISIS DE JOIN:")
print(f"   User IDs en loans Q1:     {len(user_ids_loans):,}")
print(f"   User IDs en customers:    {len(user_ids_customers):,}")
print(f"   CON match (inner join):   {len(users_con_match):,} ({len(users_con_match)/len(user_ids_loans)*100:.1f}%)")
print(f"   SIN match (left only):    {len(users_sin_match):,} ({len(users_sin_match)/len(user_ids_loans)*100:.1f}%)")

# Ver columnas disponibles en customers_raw
print(f"\n📋 COLUMNAS EN CUSTOMERS RAW:")
print(f"   {customers_raw.columns.tolist()}")

# Risk bands (esto sí existe en raw)
if 'risk_band_production' in customers_raw.columns:
    print(f"\n📈 RISK BANDS EN CUSTOMERS:")
    print(customers_raw['risk_band_production'].value_counts().sort_index())

# Si existe risk_segment, mostrarlo
if 'risk_segment' in customers_raw.columns:
    print(f"\n🎯 RISK SEGMENTS EN CUSTOMERS:")
    risk_dist = customers_raw['risk_segment'].value_counts()
    for segment, count in risk_dist.items():
        print(f"   {segment}: {count:,} ({count/len(customers_raw)*100:.1f}%)")
else:
    print(f"\n⚠️ 'risk_segment' no existe en raw - se crea en transformación desde risk_band_production")

if len(users_sin_match) == 0:
    print(f"\n✅ Todos los loans tienen customer - JOIN completo")
else:
    print(f"\n⚠️ {len(users_sin_match):,} users sin customer - usar LEFT JOIN")

4️⃣ VALIDACIÓN: JOIN LOANS ↔ CUSTOMERS

📊 ANÁLISIS DE JOIN:
   User IDs en loans Q1:     4,500
   User IDs en customers:    4,500
   CON match (inner join):   4,500 (100.0%)
   SIN match (left only):    0 (0.0%)

📋 COLUMNAS EN CUSTOMERS RAW:
   ['user_id', 'acquisition_date', 'acquisition_cost', 'channel', 'risk_band_production', 'city', 'state']

📈 RISK BANDS EN CUSTOMERS:
risk_band_production
1                  17
2                 211
3                1316
4.1               749
4.2              1104
5                1081
missing_score      20
Name: count, dtype: int64

⚠️ 'risk_segment' no existe en raw - se crea en transformación desde risk_band_production

✅ Todos los loans tienen customer - JOIN completo


---
## 5️⃣ VALIDACIÓN: JOIN LOANS ↔ REPAYMENTS
¿Cuántos loans tienen pagos? ¿Cuántos no tienen ningún pago?

In [9]:
print("="*80)
print("5️⃣ VALIDACIÓN: JOIN LOANS ↔ REPAYMENTS")
print("="*80)

# IDs
loan_ids_q1 = set(loans_q1_df['loan_id'].unique())
loan_ids_con_pagos = set(repayments_raw['loan_id'].unique())

# Intersección
loans_con_pagos = loan_ids_q1 & loan_ids_con_pagos
loans_sin_pagos = loan_ids_q1 - loan_ids_con_pagos

print(f"\n📊 ANÁLISIS DE PAGOS (Q1 2025):")
print(f"   Total loans Q1:           {len(loan_ids_q1):,}")
print(f"   CON pagos:                {len(loans_con_pagos):,} ({len(loans_con_pagos)/len(loan_ids_q1)*100:.1f}%)")
print(f"   SIN pagos:                {len(loans_sin_pagos):,} ({len(loans_sin_pagos)/len(loan_ids_q1)*100:.1f}%)")

# Estadísticas de pagos
repayments_q1 = repayments_raw[repayments_raw['loan_id'].isin(loan_ids_q1)]
pagos_por_loan = repayments_q1.groupby('loan_id').size()

print(f"\n📈 PAGOS POR LOAN (solo loans con pagos):")
print(f"   Mínimo:   {pagos_por_loan.min()}")
print(f"   Promedio: {pagos_por_loan.mean():.1f}")
print(f"   Mediana:  {pagos_por_loan.median():.0f}")
print(f"   Máximo:   {pagos_por_loan.max()}")

print(f"\n✅ LEFT JOIN preservará {len(loan_ids_q1):,} loans")
print(f"   → {len(loans_sin_pagos):,} tendrán revenue = 0 (COALESCE de NULLs)")

5️⃣ VALIDACIÓN: JOIN LOANS ↔ REPAYMENTS

📊 ANÁLISIS DE PAGOS (Q1 2025):
   Total loans Q1:           9,396
   CON pagos:                8,419 (89.6%)
   SIN pagos:                977 (10.4%)

📈 PAGOS POR LOAN (solo loans con pagos):
   Mínimo:   1
   Promedio: 3.6
   Mediana:  2
   Máximo:   25

✅ LEFT JOIN preservará 9,396 loans
   → 977 tendrán revenue = 0 (COALESCE de NULLs)


---
## 6️⃣ VALIDACIÓN: CÁLCULO DE REVENUE
revenue = interest + fees + penalty + tax_interest + tax_fees + tax_penalty

In [10]:
print("="*80)
print("6️⃣ VALIDACIÓN: CÁLCULO DE REVENUE")
print("="*80)

# Calcular componentes de revenue
repayments_q1 = repayments_raw[repayments_raw['loan_id'].isin(loan_ids_q1)].copy()

componentes = {
    'interest': repayments_q1['interestamount_trans'].sum(),
    'fees': repayments_q1['feesamount_trans'].sum(),
    'penalty': repayments_q1['penaltyamount_trans'].sum(),
    'tax_interest': repayments_q1['taxoninterestamount_trans'].sum(),
    'tax_fees': repayments_q1['taxonfeesamount_trans'].sum(),
    'tax_penalty': repayments_q1['taxonpenaltyamount_trans'].sum(),
}

revenue_total = sum(componentes.values())

print(f"\n💰 COMPONENTES DE REVENUE (Q1 2025):")
print(f"   ┌────────────────────────────────────┐")
for comp, valor in componentes.items():
    pct = valor/revenue_total*100 if revenue_total > 0 else 0
    print(f"   │ {comp:<15} ${valor:>12,.2f} ({pct:>5.1f}%) │")
print(f"   ├────────────────────────────────────┤")
print(f"   │ TOTAL           ${revenue_total:>12,.2f} (100.0%) │")
print(f"   └────────────────────────────────────┘")

# Revenue por loan
repayments_q1['revenue'] = (
    repayments_q1['interestamount_trans'] + 
    repayments_q1['feesamount_trans'] + 
    repayments_q1['penaltyamount_trans'] + 
    repayments_q1['taxoninterestamount_trans'] +
    repayments_q1['taxonfeesamount_trans'] +
    repayments_q1['taxonpenaltyamount_trans']
)

revenue_por_loan = repayments_q1.groupby('loan_id')['revenue'].sum()

print(f"\n📈 REVENUE POR LOAN:")
print(f"   Mínimo:   ${revenue_por_loan.min():,.2f}")
print(f"   Promedio: ${revenue_por_loan.mean():,.2f}")
print(f"   Mediana:  ${revenue_por_loan.median():,.2f}")
print(f"   Máximo:   ${revenue_por_loan.max():,.2f}")

print(f"\n✅ Revenue total esperado: ${revenue_total:,.2f}")

6️⃣ VALIDACIÓN: CÁLCULO DE REVENUE

💰 COMPONENTES DE REVENUE (Q1 2025):
   ┌────────────────────────────────────┐
   │ interest        $1,307,381.24 ( 72.9%) │
   │ fees            $  268,881.03 ( 15.0%) │
   │ penalty         $        0.00 (  0.0%) │
   │ tax_interest    $  180,327.87 ( 10.1%) │
   │ tax_fees        $   37,087.04 (  2.1%) │
   │ tax_penalty     $        0.00 (  0.0%) │
   ├────────────────────────────────────┤
   │ TOTAL           $1,793,677.18 (100.0%) │
   └────────────────────────────────────┘

📈 REVENUE POR LOAN:
   Mínimo:   $0.00
   Promedio: $213.05
   Mediana:  $66.00
   Máximo:   $6,131.16

✅ Revenue total esperado: $1,793,677.18


---
## 7️⃣ VALIDACIÓN: DISTRIBUCIÓN POR RISK SEGMENT
¿Cómo se distribuyen los loans y el revenue por segmento de riesgo?

In [14]:
print("="*80)
print("7️⃣ VALIDACIÓN: DISTRIBUCIÓN POR RISK SEGMENT")
print("="*80)

# Primero crear risk_segment desde risk_band_production si no existe
if 'risk_segment' not in customers_raw.columns:
    # Convertir a numérico primero
    customers_raw['risk_band_numeric'] = pd.to_numeric(customers_raw['risk_band_production'], errors='coerce')
    
    def assign_risk_segment(band):
        if pd.isna(band):
            return 'Unknown'
        elif band <= 2:
            return 'Low Risk'
        elif band <= 3:
            return 'Medium Risk'
        else:
            return 'High Risk'
    
    customers_raw['risk_segment'] = customers_raw['risk_band_numeric'].apply(assign_risk_segment)
    print("⚠️ 'risk_segment' creado desde 'risk_band_production'")

# Columnas a usar (solo las que existen)
cols_to_merge = ['user_id']
if 'risk_segment' in customers_raw.columns:
    cols_to_merge.append('risk_segment')
if 'risk_band_production' in customers_raw.columns:
    cols_to_merge.append('risk_band_production')
if 'acquisition_cost' in customers_raw.columns:
    cols_to_merge.append('acquisition_cost')

# Crear tabla de loans con risk segment
loans_con_riesgo = loans_q1_df.merge(
    customers_raw[cols_to_merge],
    on='user_id',
    how='left'
)

# Agregar revenue por loan
revenue_df = repayments_q1.groupby('loan_id')['revenue'].sum().reset_index()
loans_con_riesgo = loans_con_riesgo.merge(revenue_df, on='loan_id', how='left')
loans_con_riesgo['revenue'] = loans_con_riesgo['revenue'].fillna(0)

# Rellenar NULLs de risk_segment
loans_con_riesgo['risk_segment'] = loans_con_riesgo['risk_segment'].fillna('Unknown')

# Rellenar NULLs de acquisition_cost si existe
if 'acquisition_cost' in loans_con_riesgo.columns:
    loans_con_riesgo['acquisition_cost'] = loans_con_riesgo['acquisition_cost'].fillna(0)
else:
    loans_con_riesgo['acquisition_cost'] = 0

# Agregar por risk segment
por_segmento = loans_con_riesgo.groupby('risk_segment').agg({
    'loan_id': 'count',
    'revenue': 'sum',
    'acquisition_cost': 'sum'
}).rename(columns={'loan_id': 'loans'})

por_segmento['revenue_per_loan'] = por_segmento['revenue'] / por_segmento['loans']
por_segmento['cac_per_loan'] = por_segmento['acquisition_cost'] / por_segmento['loans']
por_segmento['pct_loans'] = por_segmento['loans'] / por_segmento['loans'].sum() * 100

print(f"\n📊 MÉTRICAS POR RISK SEGMENT:")
display(por_segmento.round(2))

print(f"\n🔍 INSIGHTS:")
print(f"   Segmento con más loans:   {por_segmento['loans'].idxmax()}")
print(f"   Segmento con más revenue: {por_segmento['revenue'].idxmax()}")
print(f"   Mayor revenue/loan:       {por_segmento['revenue_per_loan'].idxmax()}")

print(f"\n✅ Distribución por segmento validada")

7️⃣ VALIDACIÓN: DISTRIBUCIÓN POR RISK SEGMENT
⚠️ 'risk_segment' creado desde 'risk_band_production'

📊 MÉTRICAS POR RISK SEGMENT:


,loans,revenue,acquisition_cost,revenue_per_loan,cac_per_loan,pct_loans
risk_segment,,,,,,
High Risk,5991,1185248.85,210804.22,197.84,35.19,63.76
Low Risk,567,51653.60,43088.59,91.10,75.99,6.03
Medium Risk,2785,546580.39,151489.76,196.26,54.39,29.64
Unknown,53,10194.34,1523.41,192.35,28.74,0.56



🔍 INSIGHTS:
   Segmento con más loans:   High Risk
   Segmento con más revenue: High Risk
   Mayor revenue/loan:       High Risk

✅ Distribución por segmento validada


---
## 8️⃣ VALIDACIÓN: AGREGACIÓN POR COHORT (VINTAGE × RISK)
¿La suma de todos los cohorts coincide con el total?

In [15]:
print("="*80)
print("8️⃣ VALIDACIÓN: AGREGACIÓN POR COHORT")
print("="*80)

# Agregar por vintage_month × risk_segment
loans_con_riesgo['vintage_month'] = loans_con_riesgo['disbursed_date'].dt.to_period('M')

cohorts = loans_con_riesgo.groupby(['vintage_month', 'risk_segment']).agg({
    'loan_id': 'count',
    'revenue': 'sum'
}).rename(columns={'loan_id': 'loans'}).reset_index()

print(f"\n📊 COHORTS (Vintage × Risk Segment):")
display(cohorts)

# Validar suma
suma_loans_cohorts = cohorts['loans'].sum()
suma_revenue_cohorts = cohorts['revenue'].sum()
total_loans_directo = len(loans_q1_df)
total_revenue_directo = revenue_total

print(f"\n🔍 VALIDACIÓN DE TOTALES:")
print(f"   LOANS:")
print(f"      Suma cohorts: {suma_loans_cohorts:,}")
print(f"      Total directo: {total_loans_directo:,}")
print(f"      Match: {suma_loans_cohorts == total_loans_directo}")

print(f"\n   REVENUE:")
print(f"      Suma cohorts:  ${suma_revenue_cohorts:,.2f}")
print(f"      Total directo: ${total_revenue_directo:,.2f}")
print(f"      Diferencia:    ${abs(suma_revenue_cohorts - total_revenue_directo):,.2f}")

if suma_loans_cohorts == total_loans_directo and abs(suma_revenue_cohorts - total_revenue_directo) < 1:
    print(f"\n✅ Agregación por cohorts CORRECTA")
else:
    print(f"\n❌ Diferencia en agregación")

8️⃣ VALIDACIÓN: AGREGACIÓN POR COHORT

📊 COHORTS (Vintage × Risk Segment):


,vintage_month,risk_segment,loans,revenue
0,2025-01,High Risk,1457,258500.46
1,2025-01,Low Risk,103,8808.08
2,2025-01,Medium Risk,571,107121.92
3,2025-01,Unknown,9,1896.14
4,2025-02,High Risk,2368,496462.91
5,2025-02,Low Risk,224,17472.51
6,2025-02,Medium Risk,1080,231028.51
7,2025-02,Unknown,22,5218.26
8,2025-03,High Risk,2166,430285.48
9,2025-03,Low Risk,240,25373.01



🔍 VALIDACIÓN DE TOTALES:
   LOANS:
      Suma cohorts: 9,396
      Total directo: 9,396
      Match: True

   REVENUE:
      Suma cohorts:  $1,793,677.18
      Total directo: $1,793,677.18
      Diferencia:    $0.00

✅ Agregación por cohorts CORRECTA


---
## 9️⃣ VALIDACIÓN: COMPARAR CON EXPORTS
¿Los archivos exportados coinciden con nuestros cálculos?

In [17]:
print("="*80)
print("9️⃣ VALIDACIÓN: COMPARAR CON EXPORTS")
print("="*80)

# Cargar exports
loans_export = pd.read_excel(exports_path / 'loans.xlsx')
customers_export = pd.read_excel(exports_path / 'customers.xlsx')

print(f"\n📋 COLUMNAS EN LOANS EXPORT:")
print(f"   {loans_export.columns.tolist()}")

# Buscar columna de fecha (más flexible)
date_cols = [c for c in loans_export.columns if any(x in c.lower() for x in ['date', 'disburs', 'created', 'month', 'vintage'])]
print(f"\n📅 Posibles columnas de fecha: {date_cols}")

if len(date_cols) > 0:
    date_col = date_cols[0]
    loans_export[date_col] = pd.to_datetime(loans_export[date_col], errors='coerce')
    loans_export['vintage_month'] = loans_export[date_col].dt.to_period('M')
    loans_export_q1 = loans_export[loans_export['vintage_month'].isin(Q1_PERIODS)]
    
    print(f"\n📊 COMPARACIÓN LOANS Q1 2025:")
    print(f"   Calculado (raw):  {len(loans_q1_df):,} loans")
    print(f"   En export:        {len(loans_export_q1):,} loans")
    print(f"   Match: {len(loans_q1_df) == len(loans_export_q1)}")
else:
    print(f"\n⚠️ No se encontró columna de fecha")
    loans_export_q1 = loans_export  # usar todo

# Buscar columna de revenue
rev_cols = [c for c in loans_export_q1.columns if 'revenue' in c.lower()]
if len(rev_cols) > 0:
    rev_col = rev_cols[0]
    revenue_export = loans_export_q1[rev_col].sum()
    
    print(f"\n💰 COMPARACIÓN REVENUE Q1 2025:")
    print(f"   Calculado (raw):  ${revenue_total:,.2f}")
    print(f"   En export:        ${revenue_export:,.2f}")
    print(f"   Diferencia:       ${abs(revenue_total - revenue_export):,.2f}")
    
    if abs(revenue_total - revenue_export) < 1:
        print(f"\n✅ Revenue COINCIDE")
    else:
        print(f"\n⚠️ Diferencia en revenue")
else:
    print(f"\n⚠️ No se encontró columna de revenue en export")

9️⃣ VALIDACIÓN: COMPARAR CON EXPORTS

📋 COLUMNAS EN LOANS EXPORT:
   ['loan_id', 'user_id', 'vintage_date', 'vintage_month', 'vintage_year', 'Total Loan Amount', 'loan_term', 'interest_rate', 'loan_sequence_number', 'flg_first_loan_customer', 'flg_recurrent_customer', 'First Loans Count', 'Repeat Loans Count', 'risk_segment_customer', 'risk_band_production_customer', 'channel_customer', 'state_customer', 'city_customer', 'tiie_28_rate', 'spread_rate', 'funding_cost_annual', 'funding_cost_monthly', 'funding_cost_daily', 'Revenue Total', 'revenue', 'Interest Total', 'interest_revenue', 'Fees Total', 'fee_revenue', 'Penalties Total', 'penalty_revenue', 'Funding Cost Total', 'funding_cost', 'Credit Loss Total', 'credit_loss', 'Cogs Total', 'cogs', 'Cac Total', 'cac', 'Financial Margin Total', 'financial_margin', 'Contribution Margin Total', 'contribution_margin', 'Net Profit Total', 'net_profit', 'principal_repaid', 'recovery_rate', 'delinquency_status', 'is_delinquent', 'dpd_bucket', 'cap

In [18]:
print("="*80)
print("🔍 INVESTIGANDO DIFERENCIA DE REVENUE")
print("="*80)

# Ver las columnas de revenue disponibles
rev_cols = [c for c in loans_export_q1.columns if 'revenue' in c.lower()]
print(f"\n📋 COLUMNAS DE REVENUE EN EXPORT:")
for col in rev_cols:
    total = loans_export_q1[col].sum()
    print(f"   {col}: ${total:,.2f}")

# Comparar componentes
print(f"\n📊 COMPARACIÓN DE COMPONENTES:")
print(f"\n   CALCULADO DESDE RAW:")
print(f"   Interest:     ${componentes['interest']:>12,.2f}")
print(f"   Fees:         ${componentes['fees']:>12,.2f}")
print(f"   Penalty:      ${componentes['penalty']:>12,.2f}")
print(f"   Tax Interest: ${componentes['tax_interest']:>12,.2f}")
print(f"   Tax Fees:     ${componentes['tax_fees']:>12,.2f}")
print(f"   Tax Penalty:  ${componentes['tax_penalty']:>12,.2f}")
print(f"   TOTAL:        ${revenue_total:>12,.2f}")

print(f"\n   EN EXPORT:")
int_cols = [c for c in loans_export_q1.columns if 'interest' in c.lower()]
fee_cols = [c for c in loans_export_q1.columns if 'fee' in c.lower()]
pen_cols = [c for c in loans_export_q1.columns if 'penal' in c.lower()]

for col in int_cols:
    print(f"   {col}: ${loans_export_q1[col].sum():,.2f}")
for col in fee_cols:
    print(f"   {col}: ${loans_export_q1[col].sum():,.2f}")
for col in pen_cols:
    print(f"   {col}: ${loans_export_q1[col].sum():,.2f}")

# ¿El export incluye taxes?
print(f"\n💡 HIPÓTESIS:")
print(f"   Diferencia: ${revenue_total - loans_export_q1['revenue'].sum():,.2f}")
taxes_total = componentes['tax_interest'] + componentes['tax_fees'] + componentes['tax_penalty']
print(f"   Total Taxes: ${taxes_total:,.2f}")
print(f"   ¿Diferencia ≈ Taxes? {abs((revenue_total - loans_export_q1['revenue'].sum()) - taxes_total) < 1000}")

🔍 INVESTIGANDO DIFERENCIA DE REVENUE

📋 COLUMNAS DE REVENUE EN EXPORT:
   Revenue Total: $1,576,262.27
   revenue: $1,576,262.27
   interest_revenue: $1,307,381.24
   fee_revenue: $268,881.03
   penalty_revenue: $0.00

📊 COMPARACIÓN DE COMPONENTES:

   CALCULADO DESDE RAW:
   Interest:     $1,307,381.24
   Fees:         $  268,881.03
   Penalty:      $        0.00
   Tax Interest: $  180,327.87
   Tax Fees:     $   37,087.04
   Tax Penalty:  $        0.00
   TOTAL:        $1,793,677.18

   EN EXPORT:
   interest_rate: $2,297.15
   Interest Total: $1,307,381.24
   interest_revenue: $1,307,381.24
   Fees Total: $268,881.03
   fee_revenue: $268,881.03
   Penalties Total: $0.00
   penalty_revenue: $0.00

💡 HIPÓTESIS:
   Diferencia: $217,414.91
   Total Taxes: $217,414.91
   ¿Diferencia ≈ Taxes? True


---
## 📋 RESUMEN FINAL

In [19]:
print("="*80)
print("📋 RESUMEN DE VALIDACIONES")
print("="*80)

print(f"""
┌─────────────────────────────────────────────────────────────────┐
│                    PIPELINE DE TRANSFORMACIÓN                   │
├─────────────────────────────────────────────────────────────────┤
│ 1. RAW CARGADO                                                  │
│    • loans:      {len(loans_raw):>10,} filas                              │
│    • repayments: {len(repayments_raw):>10,} filas                              │
│    • customers:  {len(customers_raw):>10,} filas                              │
├─────────────────────────────────────────────────────────────────┤
│ 2. DEDUPLICACIÓN                                                │
│    • Snapshots eliminados: {snapshots_duplicados:>10,}                        │
│    • Loans únicos:         {loans_unicos:>10,}                        │
├─────────────────────────────────────────────────────────────────┤
│ 3. FILTRO Q1 2025                                               │
│    • Loans Q1:    {loans_q1:>10,}                                     │
│    • Excluidos:   {loans_otros:>10,}                                     │
├─────────────────────────────────────────────────────────────────┤
│ 4. JOIN CUSTOMERS                                               │
│    • Con match:   {len(users_con_match):>10,}                                     │
│    • Sin match:   {len(users_sin_match):>10,}                                     │
├─────────────────────────────────────────────────────────────────┤
│ 5. JOIN REPAYMENTS                                              │
│    • Con pagos:   {len(loans_con_pagos):>10,}                                     │
│    • Sin pagos:   {len(loans_sin_pagos):>10,}                                     │
├─────────────────────────────────────────────────────────────────┤
│ 6. REVENUE CALCULADO                                            │
│    • Total: ${revenue_total:>15,.2f}                               │
├─────────────────────────────────────────────────────────────────┤
│ 7. COHORTS                                                      │
│    • Total cohorts: {len(cohorts):>10,}                                    │
│    • Suma = Total:  ✓                                           │
└─────────────────────────────────────────────────────────────────┘

✅ TODAS LAS VALIDACIONES COMPLETADAS
""")

📋 RESUMEN DE VALIDACIONES

┌─────────────────────────────────────────────────────────────────┐
│                    PIPELINE DE TRANSFORMACIÓN                   │
├─────────────────────────────────────────────────────────────────┤
│ 1. RAW CARGADO                                                  │
│    • loans:         202,294 filas                              │
│    • repayments:     91,296 filas                              │
│    • customers:       4,500 filas                              │
├─────────────────────────────────────────────────────────────────┤
│ 2. DEDUPLICACIÓN                                                │
│    • Snapshots eliminados:    173,072                        │
│    • Loans únicos:             29,222                        │
├─────────────────────────────────────────────────────────────────┤
│ 3. FILTRO Q1 2025                                               │
│    • Loans Q1:         9,396                                     │
│    • Excluidos:       19,82